In [0]:
import urllib.request
import zipfile

# download and unzip dataset
url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
# make sure directory (/dbfs/FileStore/datasets) is already created in your dbfs
data_save_path = "/dbfs/FileStore/datasets/ml-latest-small.zip"
urllib.request.urlretrieve(url, data_save_path)

dist_path = "/dbfs/FileStore/datasets/"
with zipfile.ZipFile(data_save_path, 'r') as zip_ref:
    zip_ref.extractall(dist_path)

In [0]:
import math
import argparse
import os
import random

from bigdl.orca import init_orca_context, stop_orca_context, OrcaContext
from bigdl.orca.learn.tf2.estimator import Estimator
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType


def build_model(num_users, num_items, class_num, layers=[20, 10], include_mf=True, mf_embed=20):
    import tensorflow as tf
    from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, concatenate, multiply

    num_layer = len(layers)
    user_input = Input(shape=(1,), dtype='int32', name='user_input')
    item_input = Input(shape=(1,), dtype='int32', name='item_input')

    mlp_embed_user = Embedding(input_dim=num_users, output_dim=int(layers[0] / 2),
                               input_length=1)(user_input)
    mlp_embed_item = Embedding(input_dim=num_items, output_dim=int(layers[0] / 2),
                               input_length=1)(item_input)

    user_latent = Flatten()(mlp_embed_user)
    item_latent = Flatten()(mlp_embed_item)

    mlp_latent = concatenate([user_latent, item_latent], axis=1)
    for idx in range(1, num_layer):
        layer = Dense(layers[idx], activation='relu',
                      name='layer%d' % idx)
        mlp_latent = layer(mlp_latent)

    if include_mf:
        mf_embed_user = Embedding(input_dim=num_users,
                                  output_dim=mf_embed,
                                  input_length=1)(user_input)
        mf_embed_item = Embedding(input_dim=num_items,
                                  output_dim=mf_embed,
                                  input_length=1)(item_input)
        mf_user_flatten = Flatten()(mf_embed_user)
        mf_item_flatten = Flatten()(mf_embed_item)

        mf_latent = multiply([mf_user_flatten, mf_item_flatten])
        concated_model = concatenate([mlp_latent, mf_latent], axis=1)
        prediction = Dense(class_num, activation='softmax', name='prediction')(concated_model)
    else:
        prediction = Dense(class_num, activation='softmax', name='prediction')(mlp_latent)

    model = tf.keras.Model([user_input, item_input], prediction)
    return model

In [0]:
lr = 0.001
epochs = 5
batch_size = 8000
backend = "ray" # ray or spark
data_dir = './'
model_dir = "/dbfs/FileStore/model/ncf_train/"
save_path = model_dir + "ncf.h5"

In [0]:
sc = init_orca_context(cluster_mode="spark-submit")

spark = OrcaContext.get_spark_session()

In [0]:
schema = StructType([StructField("user", IntegerType(), False),
                     StructField("item", IntegerType(), False),
                     StructField("label", FloatType(), False)])

data = spark.read.csv("dbfs:/FileStore/datasets/ml-latest-small/ratings.csv", schema, header=True)

max_user_id = data.agg({"user": "max"}).collect()[0]["max(user)"]
max_item_id = data.agg({"item": "max"}).collect()[0]["max(item)"]

num_users = max_user_id + 1
num_items = max_item_id + 1
class_num = data.select('label').distinct().count()

train, test = data.randomSplit([0.8, 0.2], seed=1)

config = {"lr": lr}


def model_creator(config):
    import tensorflow as tf

    model = build_model(num_users, num_items, class_num)
    print(model.summary())
    optimizer = tf.keras.optimizers.Adam(config["lr"])
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['sparse_categorical_crossentropy', 'accuracy'])
    return model


steps_per_epoch = math.ceil(train.count() / batch_size)
val_steps = math.ceil(test.count() / batch_size)

estimator = Estimator.from_keras(model_creator=model_creator,
                                 verbose=False,
                                 config=config,
                                 backend=backend,
                                 model_dir=model_dir)
estimator.fit(train,
              batch_size=batch_size,
              epochs=epochs,
              feature_cols=['user', 'item'],
              label_cols=['label'],
              steps_per_epoch=steps_per_epoch,
              validation_data=test,
              validation_steps=val_steps)

predictions = estimator.predict(test,
                                batch_size=batch_size,
                                feature_cols=['user', 'item'])
print("Predictions on validation dataset:")
predictions.show(5, truncate=False)

print("Saving model to: ", save_path)
estimator.save(save_path)

# load with estimator.load(save_path)
# estimator.load(save_path)

In [0]:
stop_orca_context()